## ComfyUI起動

In [ ]:
!pip install python-dotenv

import os
import socket
import time
from dotenv import load_dotenv

def wait_for_port(host, port, timeout=20):
    start_time = time.time()
    while time.time() - start_time < timeout:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            try:
                s.connect((host, port))
                return True
            except (ConnectionRefusedError, OSError):
                time.sleep(0.5)
    return False

# よく使うVAEをダウンロード
%cd ~/ComfyUI/models/vae
!wget -nc 'https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors'

# 2. ffxvsのネガティブプロンプトパックをクローン
%cd ~/
!git clone https://huggingface.co/ffxvs/negative-prompts-pack
%cd negative-prompts-pack
!git lfs pull

# 3. ComfyUIのembeddingフォルダへコピー
!mkdir -p ~/ComfyUI/models/embeddings/SD1.5
!mv *.pt *.safetensors ~/ComfyUI/models/embeddings/SD1.5

# 4. cloneしたリポジトリを削除して後始末
%cd ~/
!rm -rf negative-prompts-pack

import subprocess

if CLOUD_GPU == "LightningAI":
    subprocess.Popen(["python", "ComfyUI/main.py", "--port", "7860"])
    print("✅ ComfyUI 起動完了")
    print("また10分以内に KeepAlive.ipynb を実行してください。")
    
# cloudflaredインストール
    !wget -q -nc -O cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
    !chmod +x cloudflared

    if wait_for_port("127.0.0.1", 7860):
        print("ComfyUI が起動してポートをListenしています。")
    else:
        print("ポート7860に接続できません。起動に失敗している可能性があります。")

    load_dotenv(os.path.expanduser("~/.env"))
    TOKEN = os.getenv("CLOUDFLARED_TOKEN")

    if not TOKEN:
        raise ValueError("CLOUDFLARED_TOKEN が取得できませんでした。.env を確認してください。")

    subprocess.Popen(["./cloudflared", "tunnel", "run", "--token", TOKEN ])
    print("✅ cloudflared tunnel 起動完了")

elif CLOUD_GPU == "RunPod":
    subprocess.Popen(["python", "main.py", "--listen", "--port", "3000"], cwd="/ComfyUI")
